This notebook assumes that the database has been filtered and has access to this filtered dataset. 

The notebook performs the relevant denoising and preprocessing on the signals in the database. In line with literature this notebook has provides functionality for both the butterworth filter and the Discrete Wavelet Transfrom (DWT) as methods to remove noise and baseline wander from the raw data. 

There is also the option to check and invert signals using the `ecg_invert` function provided by Neurokit2. This takes a while and provided minimal change in final results in binary classification so was not done included in final preprocessing.

The notebook results in an n_channels * sample_length array of denoised signals through the chosen method.

In [1]:
%run "Database_Filtering.ipynb" #allowing access to the filtered database

Filtering Database


100%|██████████| 549/549 [00:58<00:00,  9.39it/s]

221 remaining out of 290


In [2]:
from scipy.signal import butter, filtfilt #for the butterworth filter
import pywt #for DWT
import neurokit2 as nk #for ecg invert

Here we can set which preprocessing methods we would like to use.

In [3]:
invert_signals = False
use_DWT = True
use_Butter = False
normalise_state = True

if use_Butter:
    preprocessing_state = "Butterworth Filter"
if use_DWT:
    preprocessing_state = "DWT"

In [4]:
print(f"normalising and preproccessing signals using {preprocessing_state}")

normalising and preproccessing signals using DWT


First we have to access the signals from the filtered patient database.

In [5]:
#accessing signals within the allowed list of patients
channel_list = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6']
no_channels = len(channel_list)
no_patients = allowed_patients.count_patients()
health_state = allowed_patients.get_diagnoses()

#looping over all patients in database
signals = np.zeros(shape=(no_patients, no_channels, sample_length))
for i in tqdm(range(0, no_patients)):
    #looping over each channel investigated
    channels = np.zeros(shape=(no_channels, sample_length))
    for j, channel in enumerate(channel_list):
        signal = allowed_patients.get_patients(i).get_signal()[:, j]
        #signal = allowed_patients.get_patients(i).read_signal([channel]).reshape(-1)

        #channels[j] = signal
    
        #flips signals if in need of inverting
        if invert_signals:
            if not np.isnan(signal).all():
                inversion_corrected_signal = nk.ecg_invert(signal)[0]
                channels[j] = inversion_corrected_signal
            else:
                channels[j] = signal
        else:
            channels[j] = signal
    
    signals[i] = channels

100%|██████████| 221/221 [00:01<00:00, 190.25it/s]


Defining preprocessing functions

In [6]:
#high-pass butterworth filter removes baseline wander
def butter_highpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

#band-pass butterworth filter to remove noise
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def denoise(signal, butter=False, DWT=False, nk_filt=False, normalise=False):
    
    sample_freq = 1000
    
    if butter and DWT:
        print('only one filter can be applied to the data')
        return None
        
    if butter:
        cutoff_frequency = 0.5  # Cutoff frequency in Hz (remove frequencies below this)
        b, a = butter_highpass(cutoff_frequency, sample_freq)
        baseline_removed_signal = filtfilt(b, a, signal)
        
        lowcut = 1 # Lower bound of the band-pass filter
        highcut = 50  # Upper bound of the band-pass filter
        b, a = butter_bandpass(lowcut, highcut, sample_freq)
        denoised_signal = filtfilt(b, a, baseline_removed_signal)
        
    elif DWT:
        coeffs = pywt.wavedec(signal, wavelet='db4')
        set_to_zero = [0, 1, 2, 3, 4]
        level_to_zero = 9
        for i in range(0, len(coeffs)):
            if i in set_to_zero or i > level_to_zero:
                coeffs[i] = np.zeros_like(coeffs[i])
        denoised_signal = pywt.waverec(coeffs, wavelet='db4')
        
    if normalise:
        norm_denoised_signal = (denoised_signal - denoised_signal.min())/(denoised_signal.max()-denoised_signal.min())
        return norm_denoised_signal
    return denoised_signal
    

Denoising the signals using desired methods.

In [8]:
denoised_signals = np.zeros(shape = signals.shape)
for i, signal in enumerate(signals):
    for j in range(0, no_channels):
        denoised_signals[i][j] = denoise(signal[j], butter=use_Butter, DWT=use_DWT, normalise=normalise_state)